In [1]:
from lib.diffusion import StableDiffusionModel
from PIL import Image

In [ ]:
try:
    photo_to_manipulate = Image.open('source_data/stefan_photo.jpg').convert("RGB")
except FileNotFoundError:
    print("Error: Image file not found. Check the file path.")
    exit(1)
except Exception as e:
    print(f"Error loading image: {e}")
    exit(1)

# diff_model = StableDiffusionModel()

# modified_image = diff_model.image_to_image(init_image=photo_to_manipulate, prompt="change the hair color to light blue")

# if modified_image:
#     modified_image.save('output/stefan_diffusion.jpg')